In [4]:
import pandas as pd
import requests
import math
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

nasdaq_data = pd.read_csv("nasdaq_screener_1647820205288.csv")
nasdaq_data.rename(columns = {"Symbol":"ticker"}, inplace = True)
nasdaq_data=nasdaq_data[["ticker", "Market Cap", "IPO Year", "Sector", "Industry"]]
nasdaq_data.head()


# Set some constant variables, I could put all of this in a seperate config file
ALPACA_API_KEY = 'Kp86d5q9_UK8PWEegj9t6NQ3pOAi6O95'
START_DATE = '2021-01-01'
END_DATE = '2021-01-01'

page = 1
# URL for all the tickers on Polygon
POLYGON_TICKERS_URL = f"https://api.polygon.io/v3/reference/tickers?limit=10000&apiKey={ALPACA_API_KEY}"
# URL FOR PRICING DATA - Note, getting pricing that is UNADJUSTED for splits, I will try and adjust those manually
POLYGON_AGGS_URL = 'https://api.polygon.io/v2/aggs/ticker/{}/range/1/day/{}/{}?unadjusted=true&apiKey={}'
# URL FOR DIVIDEND DATA
POLYGON_DIV_URL = 'https://api.polygon.io/v2/reference/dividends/{}?apiKey={}'
# URL FOR STOCK SPLITS
POLYGON_SPLIT_URL = 'https://api.polygon.io/v2/reference/splits/{}?apiKey={}'
#URL FOR TICKER TYPES
POLYGON_TYPES_URL = 'https://api.polygon.io/v2/reference/types?apiKey={}'

#%% 
# Get the list of all supported tickers from Polygon.io
def get_ticker_data(url = POLYGON_TICKERS_URL):
    
    # Initial request to get the ticker count
    data = requests.get(url).json()
    all_tickers = [data['results'][i] for i in range(0, len(data['results']))]  
    df = pd.DataFrame(all_tickers)    
    
    while True:
        try:
            next_url = data['next_url']
            next_page_url = f"{next_url}&apiKey={ALPACA_API_KEY}"
            data = requests.get(next_page_url).json()
            next_page_data = [data['results'][i] for i in range(0, len(data['results']))]  
            df = pd.concat([df, pd.DataFrame(next_page_data)], axis = 0)
        except:
            print("no more pages to iterate throguh")
            break
    
    return df
        
all_tickers = get_ticker_data()

# join polygon ticker data with nasdaq ticker data to get industries and sectors included
all_tickers = all_tickers.merge(nasdaq_data, how = "left", on = "ticker")
tickers_list = all_tickers["ticker"].unique()





no more pages to iterate throguh


## Get NASDAQ data for additional info later

In [ ]:
nasdaq_data = pd.read_csv("nasdaq_screener_1647820205288.csv")
nasdaq_data.rename(columns = {"Symbol":"ticker"}, inplace = True)
nasdaq_data=nasdaq_data[["ticker", "Market Cap", "IPO Year", "Sector", "Industry"]]
nasdaq_data.head()

## Get tickers data from polygon

In [ ]:
# Set some constant variables, I could put all of this in a seperate config file
ALPACA_API_KEY = 'Kp86d5q9_UK8PWEegj9t6NQ3pOAi6O95'
START_DATE = '2021-01-01'
END_DATE = '2021-01-01'

page = 1
# URL for all the tickers on Polygon
POLYGON_TICKERS_URL = f"https://api.polygon.io/v3/reference/tickers?limit=10000&apiKey={ALPACA_API_KEY}"
# URL FOR PRICING DATA - Note, getting pricing that is UNADJUSTED for splits, I will try and adjust those manually
POLYGON_AGGS_URL = 'https://api.polygon.io/v2/aggs/ticker/{}/range/1/day/{}/{}?unadjusted=true&apiKey={}'
# URL FOR DIVIDEND DATA
POLYGON_DIV_URL = 'https://api.polygon.io/v2/reference/dividends/{}?apiKey={}'
# URL FOR STOCK SPLITS
POLYGON_SPLIT_URL = 'https://api.polygon.io/v2/reference/splits/{}?apiKey={}'
#URL FOR TICKER TYPES
POLYGON_TYPES_URL = 'https://api.polygon.io/v2/reference/types?apiKey={}'

#%% 
# Get the list of all supported tickers from Polygon.io
def get_ticker_data(url = POLYGON_TICKERS_URL):
    
    # Initial request to get the ticker count
    data = requests.get(url).json()
    all_tickers = [data['results'][i] for i in range(0, len(data['results']))]  
    df = pd.DataFrame(all_tickers)    
    
    while True:
        try:
            next_url = data['next_url']
            next_page_url = f"{next_url}&apiKey={ALPACA_API_KEY}"
            data = requests.get(next_page_url).json()
            next_page_data = [data['results'][i] for i in range(0, len(data['results']))]  
            df = pd.concat([df, pd.DataFrame(next_page_data)], axis = 0)
        except:
            print("no more pages to iterate throguh")
            break
    
    return df
        
all_tickers = get_ticker_data()
all_tickers.to_csv("all_tickers_list.csv")

ciks = list(all_tickers[(all_tickers['locale']=='us')&(all_tickers["active"]==True)]["cik"].unique())
pd.DataFrame(ciks).to_csv("filtered_ciks.csv")
len(ciks)

## Join ticker data with NASDAQ data

## Get Financials data

In [319]:
def get_financials(ciks=ciks, tickers_list=tickers_list,
                  period_of_report_date = "2021-01-01",
                  limit = 100,
                  api_key = "Kp86d5q9_UK8PWEegj9t6NQ3pOAi6O95",
                  timeframe = "annual",
                  financials_url_prefix = "https://api.polygon.io/vX/reference/financials"
                  ):
    df = pd.DataFrame()
    data = {}
    
    start_date = []
    end_date = []
    fiscal_year = []
    income_statement = pd.DataFrame()
    balance_sheet = pd.DataFrame()
    comprehensive_income = pd.DataFrame()
    cash_flow_statement = pd.DataFrame()
    fin_metadata_df = pd.DataFrame()
    
    count = 1
    # Initial request
    for cik in ciks:
#     financial_url = f"{financials_url_prefix}?ticker={ticker}&period_of_report_date={period_of_report_date}&timeframe={timeframe}&limit={limit}&apiKey={api_key}"
        count+=1
        financial_url = f"https://api.polygon.io/vX/reference/financials?cik={cik}&period_of_report_date.gte={period_of_report_date}&timeframe=annual&limit=100&apiKey=JzDlY2BqXVFHBSBGT14xfKjFbAHI8nNo"    
        data = requests.get(financial_url).json()
        if data['results']!=[]:
            fin_metadata = pd.DataFrame(data['results'])
            fin_metadata_df = fin_metadata_df.append(fin_metadata)
        else:
            continue
        if count%100==0:
            print(count)
#     fin_metadata.drop(columns = "financials", inplace = True)


# def my_func(val):
#     if isinstance(val,dict):
#         return val['value']
#     else:
#         return val

# for col in cols:
#     df[col]=df[col].apply(my_func)
    
    return data, fin_metadata_df

data, fin_metadata = get_financials()

In [72]:
fin_metadata.shape

(4035, 10)

In [49]:
balance_sheet = packed_financial_data['balance_sheet'].apply(pd.Series)
income_statement = packed_financial_data['income_statement'].apply(pd.Series)
comprehensive_income = packed_financial_data['comprehensive_income'].apply(pd.Series)
cash_flow_statement = packed_financial_data['cash_flow_statement'].apply(pd.Series)

unpacked_fin_data = pd.concat([fin_metadata, packed_financial_data, balance_sheet, income_statement, comprehensive_income, cash_flow_statement], axis = 1)
unpacked_fin_data.drop(columns = ["balance_sheet", "comprehensive_income", "income_statement", "cash_flow_statement", "financials"], inplace = True)



In [50]:
# extract values from columns of dictionaries
def my_func(val):
    if isinstance(val,dict):
        return val['value']
    else:
        return val

for col in cols:
    unpacked_fin_data[col]=unpacked_fin_data[col].apply(my_func)

fin_metadata.rename(columns = {"cik":"cik_dupe", 
                               "start_date":"start_date_dupe", 
                               "end_date":"end_date_dupe"}, inplace = True)

fin_df = pd.concat([fin_metadata, unpacked_fin_data], axis = 1)

for i in fin_df.columns:
    if "dupe" in i:
        df.drop(columns = i, inplace = True)
        
all_tickers = all_tickers.drop_duplicates(subset = "cik")
all_tickers.to_csv("filtered_correct_tickers.csv")

fin_df.to_csv("fin_data_cleaned.csv")

## Join financials with ticker data

In [ ]:
# join financial data with other ticker data
df = fin_df[fin_df["cik"].notnull()].merge(all_tickers[all_tickers["cik"].notnull()], how = "left", on = 'cik')

## Get start date stock prices

In [291]:
start_date_cols =  {"v":"start_date_volume", 
         "c":"start_date_close_price", 
         "o":"start_date_open_price",
           "h": "start_date_high_price", 
           "l":"start_date_low_proce"}


from utils import add_days, subtract_days
import numpy as np

df["start_date"].astype(str)
df["start_date_upper_limit"]=df['start_date'].apply(add_days)


df["start_date"].astype(str)
df["start_date_lower_limit"]=df["start_date"].apply(subtract_days)


# create a dictionary of tickers, start_dates and start_dates to pass to api_url to fetch for stock price data
values_list = []
start_date = list(df['start_date'])
start_date_upper_limit = list(df['start_date_upper_limit'])
d_start_date = pd.DataFrame()
start_date_range = list(zip(df['start_date'], df['start_date_upper_limit']))

iterable = list(zip(list(df['ticker']), start_date_range))
d_start_date = {k:v for k, v in iterable}
d_start_date

data = {}

for ticker, start_date in d_start_date.items():
    start_date_url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start_date[0]}/{start_date[1]}?adjusted=true&sort=asc&limit=120&apiKey={api_key}"
    data[ticker]=requests.get(start_date_url).json()
    
starting_price_df = pd.DataFrame(data).T
starting_price_df = starting_price_df.reset_index()


df_start_date_price = pd.DataFrame()
count = 0
ticker_list = []
for ticker in starting_price_df['ticker'].unique():
    count+=1
    try: 
        df_temp = pd.DataFrame(data[ticker]['results'][0], index = [count])
        df_start_date_price = df_start_date_price.appstart(df_temp)
        ticker_list.appstart(ticker)
    except:
        continue
df_start_date_price['ticker']=ticker_list
df_start_date_price.rename(columns = start_date_cols, inplace = True)


### Get end date stock prices

In [297]:
from utils import add_days, subtract_days

end_date_cols =  {"v":"end_date_volume", 
         "c":"end_date_close_price", 
         "o":"end_date_open_price",
           "h": "end_date_high_price", 
           "l":"end_date_low_price"}

df["end_date"].astype(str)
df["end_date_upper_limit"]=df['end_date'].apply(add_days)


df["end_date"].astype(str)
df["end_date_lower_limit"]=df["end_date"].apply(subtract_days)


# create a dictionary of tickers, end_dates and end_dates to pass to api_url to fetch for stock price data
values_list = []
end_date = list(df['end_date'])
end_date_upper_limit = list(df['end_date_upper_limit'])
d_end_date = pd.DataFrame()
end_date_range = list(zip(df['end_date'], df['end_date_upper_limit']))

iterable = list(zip(list(df['ticker']), end_date_range))
d_end_date = {k:v for k, v in iterable}
d_end_date

data = {}

for ticker, end_date in d_end_date.items():
    end_date_url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{end_date[1]}/{end_date[0]}?adjusted=true&sort=asc&limit=120&apiKey={api_key}"
    data[ticker]=requests.get(end_date_url).json()
    
ending_price_df = pd.DataFrame(data).T
ending_price_df = ending_price_df.reset_index()


df_end_date_price = pd.DataFrame()
count = 0
ticker_list = []
for ticker in ending_price_df['ticker'].unique():
    count+=1
    try: 
        df_temp = pd.DataFrame(data[ticker]['results'][0], index = [count])
        df_end_date_price = df_end_date_price.append(df_temp)
        ticker_list.append(ticker)
    except:
        continue
df_end_date_price['ticker']=ticker_list
df_end_date_price.rename(columns = end_date_cols, inplace = True)


## Cobmine and save start date and end date stock prices

In [ ]:
prices_df = df_end_date_price.merge(df_start_date_price, how = "inner", on = "ticker")
prices_df.to_csv("stock_prices_data.csv")

## Combine financials and stock price data

In [320]:
prices_df.shape

(3301, 17)

In [326]:
df.shape

(4035, 119)

In [335]:
final_df = prices_df.merge(df, how = 'inner', on = 'ticker')

final_df.to_csv("final_data_use_this.csv")
final_df.shape

(3346, 135)

In [334]:
final_df[final_df['ticker']=="TSLA"][['start_date', "start_date_open_price"]]

,start_date,start_date_open_price
3000,2021-01-01,719.46


In [340]:
final_df[['t_x', 'n_x', "t_y", 'n_y']]

,t_x,n_x,t_y,n_y
0,1610082000000,19370,1604293200000,20967
1,1610082000000,22457,1609736400000,30122
2,1610082000000,1478,1609736400000,1192
3,1610082000000,1249,1609736400000,1164
4,1610082000000,176470,1609736400000,238614
...,...,...,...,...
3341,1610082000000,4877,1580446800000,4713
3342,1610082000000,13411,1580706000000,9918
3343,1610082000000,6733,1609736400000,3769
3344,1610082000000,4751,1609736400000,4364


In [249]:
import numpy as np

# starting_price_df = pd.DataFrame(data).T
# starting_price_df = starting_price_df.reset_index()

# starting_price_data = pd.DataFrame(starting_price_df["results"])


df_start_date_price = pd.DataFrame()
count = 0
ticker_list = []
for ticker in starting_price_df['ticker'].unique():
    count+=1
    try: 
        df_temp = pd.DataFrame(data[ticker]['results'][0], index = [count])
        df_start_date_price = df_start_date_price.append(df_temp)
        ticker_list.append(ticker)
    except:
        continue
df_start_date_price['ticker']=ticker_list

In [288]:
df_start_date_price

,v,vw,o,c,h,l,t,n,ticker
1,1585993.0,104.7440,103.5200,105.2400,105.3500,103.2900,1604293200000,20967,A
2,5504234.0,23.5663,23.6200,23.2500,24.4500,23.1250,1609736400000,30122,AA
3,828215.0,19.3149,19.3300,19.3500,19.3600,19.2243,1609736400000,1192,AAAU
4,236353.0,3.7263,3.8300,3.7300,3.8400,3.6650,1609736400000,1164,AAIC
5,69732465.0,15.2377,15.8500,15.1300,15.8800,15.0300,1609736400000,238614,AAL
...,...,...,...,...,...,...,...,...,...
3374,409230.0,31.7885,32.5200,31.1700,32.5200,31.0900,1580446800000,4713,ZUMZ
3375,1224076.0,15.2747,14.7600,15.3200,15.4800,14.7600,1580706000000,9918,ZUO
3376,244852.0,48.9990,48.0000,48.9700,50.6000,47.1860,1609736400000,3769,ZYME
3377,680304.0,3.3204,3.3300,3.3250,3.3700,3.2500,1609736400000,4364,ZYNE


In [285]:
starting_price_df

,index,ticker,queryCount,resultsCount,adjusted,results,status,request_id,count,error,v,vw,o,c,h,l,t,n
0,A,A,47,47,True,"[{'v': 1585993.0, 'vw': 104.744, 'o': 103.52, ...",OK,2d7a7087dba70491c32bf88707e78ed6,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AA,AA,5,5,True,"[{'v': 5504234.0, 'vw': 23.5663, 'o': 23.62, '...",OK,a2c4e97664b381dc99d926fbcf051ec8,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAAU,AAAU,5,5,True,"[{'v': 828215, 'vw': 19.3149, 'o': 19.33, 'c':...",OK,88512b3bfbda094d8ec7073a72b8bb1a,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAIC,AAIC,5,5,True,"[{'v': 236353, 'vw': 3.7263, 'o': 3.83, 'c': 3...",OK,54ba006313af6d3a360925ca28f012f9,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAL,AAL,5,5,True,"[{'v': 69732465.0, 'vw': 15.2377, 'o': 15.85, ...",OK,65b555688b42589ecdc107ce9a66acc3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6750,ZYXI,ZYXI,5,5,True,"[{'v': 718312.1, 'vw': 12.7765, 'o': 12.3727, ...",OK,118d7968c870187272494824f272579c,5,NaN,409230.0,31.7885,32.5200,31.1700,32.5200,31.090,1.580447e+12,4713.0
6751,ZYXI,ZYXI,5,5,True,"[{'v': 718312.1, 'vw': 12.7765, 'o': 12.3727, ...",OK,118d7968c870187272494824f272579c,5,NaN,1224076.0,15.2747,14.7600,15.3200,15.4800,14.760,1.580706e+12,9918.0
6752,ZYXI,ZYXI,5,5,True,"[{'v': 718312.1, 'vw': 12.7765, 'o': 12.3727, ...",OK,118d7968c870187272494824f272579c,5,NaN,244852.0,48.9990,48.0000,48.9700,50.6000,47.186,1.609736e+12,3769.0
6753,ZYXI,ZYXI,5,5,True,"[{'v': 718312.1, 'vw': 12.7765, 'o': 12.3727, ...",OK,118d7968c870187272494824f272579c,5,NaN,680304.0,3.3204,3.3300,3.3250,3.3700,3.250,1.609736e+12,4364.0


In [268]:
starting_price_df['ticker'].unique()

array(['A', 'AA', 'AAAU', ..., 'ZYME', 'ZYNE', 'ZYXI'], dtype=object)

In [203]:
from datetime import datetime
from datetime import timedelta

def add_days(given_date):
    given_date = '2021-01-01'

# Create datetime object from date string
    date_format = '%Y-%m-%d'
    dtObj = datetime.strptime(given_date, date_format)
    n = 7
    future_date = dtObj + timedelta(days=n)
    future_date_str = future_date.strftime(date_format)
    return future_date_str

df["start_date"].astype(str)
df["start_date_upper_limit"]=df['start_date'].apply(utils.add_days)


def subtract_days(given_date):
    given_date = '2021-01-01'

# Create datetime object from date string
    date_format = '%Y-%m-%d'
    dtObj = datetime.strptime(given_date, date_format)
    n = 7
    future_date = dtObj - timedelta(days=n)
    future_date_str = future_date.strftime(date_format)
    return future_date_str

df["end_date"].astype(str)
df["end_date_lower_limit"]=df["end_date"].apply(utils.subtract_days)

In [204]:
df[["end_date", "end_date_lower_limit"]]

,end_date,end_date_lower_limit
0,2021-10-31,2020-12-25
1,2021-12-31,2020-12-25
2,2021-12-31,2020-12-25
3,2021-12-31,2020-12-25
4,2021-09-30,2020-12-25
...,...,...
4030,2021-12-31,2020-12-25
4031,2021-12-31,2020-12-25
4032,2021-12-31,2020-12-25
4033,2021-12-31,2020-12-25


In [128]:

df = df.merge(df_start_date_price, on=['ticker',"start_date"], how='left')
df = df.merge(df_end_date_price, on='ticker', how='left')
print(df.shape)
print(list(df.columns))
df.head()


0       2020-11-01
1       2021-01-01
2       2021-01-01
3       2021-01-01
4       2020-11-06
           ...    
4030    2021-01-01
4031    2021-01-01
4032    2021-01-01
4033    2021-01-01
4034    2021-01-01
Name: start_date, Length: 4035, dtype: object